In [1]:
import PySimpleGUI as sg
import os.path

import csv
import boto3
import os
import glob

from botocore.client import Config

# First the window layout in 2 columns
file_list_1=[]
file_list_2=[]
file_list_column = [

    [sg.Text("Image Folder"),
     sg.In(size=(25, 1), enable_events=True, key="-FOLDER-"),
     sg.FolderBrowse(),
     sg.Stretch(), sg.Button("Upload images to AWS Rekognition")],
    
    [sg.Listbox(values=[], enable_events=True, size=(46, 20), key="-FILE LIST 1-"),
     sg.Listbox(values=[], enable_events=True, size=(28, 20), key="-AWS LIST 1-")],
    [sg.Text("The image count is:", len(file_list_1)),
     sg.In(size=(25, 1), enable_events=True, key="-COUNT1-")]
]

layout = [[sg.Column(file_list_column)]]
window = sg.Window("Digital Forensics Cloud Image Recognition", layout=layout, finalize=True)

In [2]:
with open ('new_user_credentials.csv', 'r') as input:
    next(input)
    reader = csv.reader(input)
    for line in reader:
        access_key_id = line[2]
        secret_access_key = line[3]

rekognition = boto3.client(
    'rekognition', 
    aws_access_key_id =access_key_id, 
    aws_secret_access_key = secret_access_key, 
    region_name = 'us-west-2')

s3 = boto3.client(
    's3',
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    region_name = 'us-west-2',
    config=Config(signature_version='s3v4'))

BUCKET_NAME = "storagerek"

In [ ]:
while True:
    event, values = window.read()
    if event == "Exit" or event == sg.WIN_CLOSED:
        break
 
    if event == "-FOLDER-":
        folder = values["-FOLDER-"]
        try:
            # Get list of files in folder
            file_list_1 = [(file, root) for root, dirs, files in os.walk(folder) for file in files if file.endswith(('.jpg','.png'))]

        except:
            file_list_1 = []
    
    window["-FILE LIST 1-"].update(file_list_1)
    window["-COUNT1-"].update(len(file_list_1))
    
    if event == "Upload images to AWS Rekognition":
        aws_list_1 = []
        for file_name, root in file_list_1:
            key = file_name
            file_dest = root+"/"+file_name
            s3.upload_file(file_dest, BUCKET_NAME, key)
            response = rekognition.detect_moderation_labels(Image={'S3Object': {
            'Bucket': 'storagerek',
            'Name' : key}})
        
            try:
                if response['ModerationLabels'][1]['Name']==('Male Swimwear Or Underwear') or ('Female Swimwear Or Underwear') :
                    print (response['ModerationLabels'][0]['Name'])
                    aws_list_1.append(response['ModerationLabels'][0]['Name'])
                    window["-AWS LIST 1-"].update(aws_list_1)
            except:
                print('Np')
                aws_list_1.append("     -----     REGULAR     -----")
                window["-AWS LIST 1-"].update(aws_list_1) 

Barechested Male
Barechested Male
Barechested Male
Barechested Male
Np
Barechested Male
Np
Np
Barechested Male
Barechested Male
Np
Np
Np
Np
Np
Female Swimwear Or Underwear
Female Swimwear Or Underwear
Np
Female Swimwear Or Underwear
Female Swimwear Or Underwear
